In [9]:
import json

import numpy as np

import os
os.getcwd()
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten

In [10]:
def drawData(X,y,predictions=None):
    bu = "┌───┬───┬───┐"
    bm = "├───┼───┼───┤"
    bd = "└───┴───┴───┘"
    symbols = " XO*"
    for i, board in enumerate(X):
        for row in range(7):
            for column in range(2):
                if predictions is None and column == 1:
                    break
                if row == 0:
                    print(bu,end=" ")
                elif row == 6:
                    print(bd,end=" ")
                elif row % 2 == 0:
                    print(bm,end=" ")
                else:
                    for j in range(3):
                        k = int(j+3/2*(row-1))
                        if column == 0:
                            mark = " " + symbols[int(board[0][k]) + 2*int(board[1][k]) -int(y[i][k])] + " "
                        else:
                            mark = "{:.1f}".format(predictions[i][k])
                            if mark == "-0.0":
                                mark = "0.0"

                        print("│" + mark,end="")
                    print("│",end=" ")
            print("")



def transform(input_array):
    corners = [[1,3,9,7], [7,9,3,1]]
    edges = [[2,6,8,4], [8,6,2,4]]
    output_arrays = []
    for variant in range(2):
        for turn in range(4):
            new_array = [0,0,0,0,0,0,0,0,0]
            for position in range(1,10):
                for i in range(4):
                    if position == corners[variant][i]:
                        new_array[corners[0][(i+turn)%4]-1] = input_array[position-1]
                    elif position == edges[variant][i]:
                        new_array[edges[0][(i+turn)%4]-1] = input_array[position-1]
                    elif position == 5:
                        new_array[4] = input_array[position-1]
            output_arrays.append(new_array)
    return output_arrays

In [11]:
datafile = open('../01_data/Data_9F_Basics.json',)
dataset = json.load(datafile)
datasize = len(dataset)

layers = 2
pixels = 3 * 3

X = np.zeros([datasize, layers, pixels])
y = np.zeros([datasize, pixels])

for i, data_elem in enumerate(dataset):
    z = 0
    for p, pixel in enumerate(data_elem["board"]):
        if pixel == 0:
            z += 1
    turn = (pixels-z)%2
    
    for p, pixel in enumerate(data_elem["board"]):
        if pixel == turn+1:
            X[i,0,p] = 1.0
        elif pixel > 0:
            X[i,1,p] = 1.0
    for g, good_move in enumerate(data_elem["good_moves"]):
        y[i,good_move-1] = 1.0


print("\n\n\nRAW DATA\n")
drawData(X,y)




RAW DATA

┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│ X │   │   │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ X │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│   │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│ X │   │   │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ * │ O │ 
├───┼───┼───

In [12]:
X_enriched = np.zeros([datasize*8, layers, pixels])
y_enriched = np.zeros([datasize*8, pixels])

for i, x_elem in enumerate(X): 
    for l, layer in enumerate(x_elem):
        for t, trans in enumerate(transform(layer)):
            X_enriched[i*8+t,l] = trans
            
for i, y_elem in enumerate(y):
    for t, trans in enumerate(transform(y_elem)):
            y_enriched[i*8+t] = trans

print("\n\n\nENRICHED DATA [ordered]\n")
drawData(X_enriched,y_enriched)




ENRICHED DATA [ordered]

┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
├───┼───┼───┤ 
│ * │ * │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│ X │   │   │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │ O 

│   │   │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │ * │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ * │ O │ O │ 
├───┼───┼───┤ 
│   │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ O │ O │ * │ 
├───┼───┼───┤ 
│ X │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼─

├───┼───┼───┤ 
│ O │   │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ O │   │ 
├───┼───┼───┤ 
│ X │   │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│ X │ X │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│ X │   │   │ 
├───┼───┼───┤ 
│ * │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ * │ 
├───┼───┼───┤ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ O │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ O │   │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ O │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
├───┼───┼───┤ 
│ X │   │ 

│ X │ * │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ * │   │ O │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ O │   │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│ * │   │ O │ 
├───┼───┼───┤ 
│ X │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│   │ O │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │   │ * │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ O │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │   │ * │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│ * │   │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ 

│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴─

│ O │ * │   │ 
├───┼───┼───┤ 
│   │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ O │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│ O │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ X │   │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ O │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│ O │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ X │   │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴─

In [13]:
shuffler = np.random.permutation(datasize*8)

X_enriched = X_enriched[shuffler]
y_enriched = y_enriched[shuffler]

print("\n\n\nENRICHED DATA [shuffled]\n")
drawData(X_enriched,y_enriched)




ENRICHED DATA [shuffled]

┌───┬───┬───┐ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ O │ 
├───┼───┼───┤ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │   │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ * │ 
├───┼───┼───┤ 
│   │ O │ X │ 
├───┼───┼───┤ 
│ O │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│ O │   │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │ O │ X │ 
├───┼───┼───┤ 
│   │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │  

│   │   │   │ 
├───┼───┼───┤ 
│ * │ O │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │ X │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ * │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │   │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│ O │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
└───┴───┴─

│   │ X │ O │ 
├───┼───┼───┤ 
│ * │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ X │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │ * │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ O │ O │   │ 
├───┼───┼───┤ 
│ X │ X │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ O │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
└───┴───┴─

│   │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│ * │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│ O │ X │   │ 
├───┼───┼───┤ 
│ * │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│ * │ O │   │ 
├───┼───┼───┤ 
│ X │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │   │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│ X │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │ * │ O │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│ O │ X │   │ 
├───┼───┼───┤ 
│ * │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ 

│   │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │ O │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│ O │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│ * │   │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ O │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │ O │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ 

┌───┬───┬───┐ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │   │ * │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │   │ * │ 
├───┼───┼───┤ 
│   │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │   │   │ 
├───┼───┼───┤ 
│ O │ X │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │ O │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ O │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│ X │ * │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ * │ 
├───┼───┼───┤ 
│   │ X │ 

In [14]:
testsize = 128;
trainsize = datasize*8 - testsize;

X_train = X_enriched [:datasize*8 - testsize]
y_train = y_enriched [:datasize*8 - testsize]

X_test = X_enriched [-testsize:]
y_test = y_enriched [-testsize:]

print ("\n\n\nTRAINDATA\n")
drawData(X_train,y_train)
print ("\n\n\nTESTDATA\n")
drawData(X_test,y_test)




TRAINDATA

┌───┬───┬───┐ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ O │ 
├───┼───┼───┤ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │   │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ * │ 
├───┼───┼───┤ 
│   │ O │ X │ 
├───┼───┼───┤ 
│ O │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│ O │   │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │ O │ X │ 
├───┼───┼───┤ 
│   │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │   │ 
├───┼───┼──

├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│ O │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ * │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │   │ 
├───┼───┼───┤ 
│ X │   │   │ 
├───┼───┼───┤ 
│ * │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ * │ O │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │ O │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ O │ 
├───┼───┼───┤ 
│ X │ O │   │ 
├───┼───┼───┤ 
│ * │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │ * │ O │ 
├───┼───┼───┤ 
│ X │   │ 

├───┼───┼───┤ 
│ O │ O │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ O │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │ * │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │   │   │ 
├───┼───┼───┤ 
│   │ O │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │ X │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │ X │   │ 
├───┼───┼───┤ 
│ * │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ * │ 
├───┼───┼───┤ 
│ O │   │ X │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│   │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬─

├───┼───┼───┤ 
│ * │   │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │   │   │ 
├───┼───┼───┤ 
│   │ X │ O │ 
├───┼───┼───┤ 
│   │ O │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ O │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│ O │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │ X │   │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│   │ X │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │   │ 
├───┼───┼───┤ 
│ O │ * │   │ 
├───┼───┼───┤ 
│   │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │ O │ * │ 
├───┼───┼───┤ 
│   │ O │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │   │ 
├───┼───┼───┤ 
│ O │   │ O │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │ O │ 
├───┼───┼───┤ 
│   │ O │   │ 
├───┼───┼───┤ 
│ * │   │ 

└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │ O │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ * │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ X │ 
├───┼───┼───┤ 
│   │ X │ O │ 
├───┼───┼───┤ 
│ * │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │ * │ O │ 
├───┼───┼───┤ 
│ O │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│ O │ O │   │ 
├───┼───┼───┤ 
│   │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│   │   │ O │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │ O │ 
├───┼───┼───┤ 
│   │ * │   │ 
├───┼───┼───┤ 
│ O │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │ O │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ X │ X │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ X │ X │ 
├───┼───┼─

│ * │ X │   │ 
├───┼───┼───┤ 
│ O │   │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ O │   │   │ 
├───┼───┼───┤ 
│ * │ X │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │ * │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ O │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ O │   │   │ 
├───┼───┼───┤ 
│ X │ O │   │ 
├───┼───┼───┤ 
│   │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ X │ * │ X │ 
├───┼───┼───┤ 
│   │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│   │ * │ O │ 
├───┼───┼───┤ 
│   │ O │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │ * │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ * │   │   │ 
├───┼───┼───┤ 
│ X │ O │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│ O │   │   │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴─

│   │   │ X │ 
├───┼───┼───┤ 
│   │ * │ O │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ O │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │   │ X │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ * │ O │   │ 
├───┼───┼───┤ 
│ X │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │   │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│ O │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ * │ X │ X │ 
├───┼───┼───┤ 
│   │   │   │ 
├───┼───┼───┤ 
│ O │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │   │   │ 
├───┼───┼───┤ 
│ * │   │ O │ 
├───┼───┼───┤ 
│ X │ O │   │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │ O │   │ 
├───┼───┼───┤ 
│ X │ X │ * │ 
├───┼───┼───┤ 
│   │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│ X │ O │ O │ 
├───┼───┼───┤ 
│   │ X │   │ 
├───┼───┼───┤ 
│   │   │ * │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ X │ 
├───┼───┼───┤ 
│   │ X │ O │ 
├───┼───┼───┤ 
│ * │   │ O │ 
└───┴───┴───┘ 
┌───┬───┬───┐ 
│   │   │ * │ 
├───┼───┼───┤ 
│ O │ X │   │ 
├───┼───┼─

In [19]:
print("\n\n\nAI TRAINING\n")

print("Testsize :",testsize)
print("Trainsize:",trainsize)
print("\n")

model = Sequential()
model.add(Dense(pixels, input_shape=(layers,pixels), activation='relu',name="input"))
model.add(Flatten(name="flatten"))
model.add(Dense(90, activation='relu', name="h1"))
model.add(Dense(90, activation='relu', name="h2"))
model.add(Dense(90, activation='relu', name="h3"))
model.add(Dense(pixels, name="output"))
# model.summary ()
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse'])
model.fit (X_train, y_train, epochs=200, batch_size=30, verbose=0)

mae, mse = model.evaluate(X_train, y_train)
print('Mean squared error train: %.4f' % (mse))
mae, mse = model.evaluate(X_test, y_test)
print('Mean squared error test: %.4f' % (mse))

predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

print ("\n\n\nTEST\n")
drawData(X_test,y_test,predictions_test)




AI TRAINING

Testsize : 128
Trainsize: 472


472/472 [==============================] - 0s 714us/step
Mean squared error train: 0.0787
128/128 [==============================] - 0s 142us/step
Mean squared error test: 0.0884



TEST

┌───┬───┬───┐ ┌───┬───┬───┐ 
│   │   │ O │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ X │ * │ X │ │0.0│1.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│   │ O │   │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ ┌───┬───┬───┐ 
│ * │ X │ X │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│   │ O │   │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ O │   │   │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ ┌───┬───┬───┐ 
│ O │ O │ * │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│   │   │   │ │0.0│0.2│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ X │   │   │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ ┌───┬───┬───┐ 
│   │   │ O │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ X │   │ * │ │0.0│0.0│0.1│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│   │   │ 

│ O │ * │   │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│   │ X │   │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ O │ X │   │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ ┌───┬───┬───┐ 
│ * │ X │ X │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│   │   │   │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ O │   │ O │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ ┌───┬───┬───┐ 
│ X │ O │ O │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ * │   │   │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ X │   │   │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ ┌───┬───┬───┐ 
│   │   │ O │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ X │ X │ * │ │0.0│0.0│1.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│   │ O │   │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ ┌───┬───┬───┐ 
│ O │   │   │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ O │   │   │ │0.0│0.0│0.0│ 
├───┼───┼───┤ ├───┼───┼───┤ 
│ X │ * │ X │ │0.0│0.0│0.0│ 
└───┴───┴───┘ └───┴───┴───┘ 
┌───┬───┬───┐ 